<p><img alt="UNQ logo" height="45px" src="http://www.unq.edu.ar/images/logo_unqui_footer.png" align="left" hspace="10px" vspace="0px"></p><h1>Redes Neuronales y Lógica Difusa</h1>

# RBFN (Radial basis function networks)

Importo los datos del repositorio de la materia

In [0]:
!git clone https://github.com/redes-unq/mDataFiles.git

Importo módulos a utilizar. En este caso además de los que venimos utilizando vamos a utilizar también el método *io* del módulo [*scipy*](https://www.scipy.org/about.html), para poder tomar los datos del archivo .mat

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat

Defino constantes

In [0]:
max_epoch = 2000 # Límite de épocas de entrenamiento
hidden_n = 8 # Cantidad de neuronas en la capa oculta
out_n = 1 # Número de salidas
eta = 0.01 # Tasa de aprendizaje
error_goal = 0.05 # Detengo si RMSE es menor a error_goal

Importo los datos y muestro los encabezados

In [0]:
datosImportados = loadmat('mDataFiles/datosRBF1.mat')
print(datosImportados.keys())
x = datosImportados["x"]
yd = datosImportados["yd"]

Inicializo los parámetros

In [0]:
CENTER = np.linspace(x.min(), x.max(), hidden_n)
# SIGMA(i) is the variance for i - th center
SIGMA = 0.02 * np.ones((hidden_n, 1))

W = np.zeros((hidden_n, out_n))

MSE = np.zeros((max_epoch, 1)) # Root mean squared error

MCENTER = np.zeros((max_epoch, len(CENTER)))
MSIGMA = np.zeros((max_epoch, len(SIGMA)))
MW = np.zeros((max_epoch, len(SIGMA)))

phi = np.zeros(len(CENTER))
y = np.zeros(len(yd))
error = np.zeros(len(yd))

Calculo la red y grafico

In [0]:
nep = 1 # Número de época
objetivoAlcanzado = False
while ((nep < max_epoch) and (not objetivoAlcanzado)):
    for n in range(len(yd)): # Recorre todos los puntos x
        for i in range(len(CENTER)):
            phi[i] = np.exp(-(x[n] - CENTER[i])**2 / (2 * SIGMA[i]**2))
        y[n] = np.sum(W.transpose()*phi)
        error[n] = yd[n] - y[n]

        for i in range(len(CENTER)):
            CENTER[i] = CENTER[i] + eta * error[n] * W[i] * phi[i] / SIGMA[i]**2 * (x[n] - CENTER[i])
            W[i] = W[i] + eta * error[n] * phi[i]
            SIGMA[i] = SIGMA[i] + eta * error[n] * W[i] * phi[i] * (x[n] - CENTER[i])**2 * 2 / SIGMA[i]**3

    if (nep == 20):
        f1 = plt.figure(1)
        plt.plot(x, yd, 'o-', label = 'yd')
        plt.plot(x, y, 'r.-', label = 'y')
        f1.legend(loc='upper right')
        plt.title('Pasada número 20')
        f1.show()

    MSE[nep] = (np.absolute(error)).mean()
    MCENTER[nep] = CENTER
    MW[nep] = W.transpose()
    MSIGMA[nep] = SIGMA.transpose()

    if MSE[nep] < error_goal:
        objetivoAlcanzado = True
        print('Alcanza el objetivo en la vuelta número: %d\n' % nep)
    nep = nep + 1

f2 = plt.figure(2)
plt.plot(x, yd, 'o-', label = 'yd')
plt.plot(x, y, 'r.-', label = 'y')
f2.legend(loc = 'upper right')
plt.title('Objetivo cumplido')
f2.show()

f3, ax3 = plt.subplots(2, 2)

ax3[0, 0].plot(MSE[1: nep, -1])
ax3[0, 0].set_title('MSE')

ax3[0, 1].plot(MW[1: nep, -1])
ax3[0, 1].set_title('MW')

ax3[1, 0].plot(MCENTER[1:nep, -1])
ax3[1, 0].set_title('MCENTER')

ax3[1, 1].plot(MSIGMA[1:nep, -1])
ax3[1, 1].set_title('MSIGMA')

plt.gca().yaxis.set_minor_formatter(plt.NullFormatter())
plt.subplots_adjust(hspace=0.55, wspace=0.35)

plt.show()

# Grafico las phis
mphi = np.zeros((len(yd), len(CENTER)))
for n in range(len(yd)): # recorre todos los puntos x
    for i in range(len(CENTER)):
        mphi[n, i] = np.exp(-(x[n] - CENTER[i])**2 / (2 * SIGMA[i]**2))
    #y(n) = sum(W'.*phi)

f4 = plt.figure(4)

for i in range(len(CENTER)):
    plt.plot(x, mphi[:, i] * W[i])

plt.title('Aportes individuales')
plt.plot(x, yd, 'r.-')
f4.show()

## RBFN con centros fijos

Importo módulos a utilizar. En este caso además de los que venimos utilizando vamos a utilizar también el método *io* del módulo [*scipy*](https://www.scipy.org/about.html), para poder tomar los datos del archivo .mat

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat

Defino constantes

In [0]:
max_epoch = 2000 # Límite de épocas de entrenamiento 
hidden_n = 8 # Cantidad de neuronas en la capa oculta
out_n = 1 # Número de salidas
eta = 0.01 # Tasa de aprendizaje
error_goal = 0.05 # Detengo si RMSE es menor a error_goal

Importo los datos y muestro los encabezados

In [0]:
datosImportados = loadmat('mDataFiles/datosRBF1.mat')
print(datosImportados.keys())
x = datosImportados["x"]
yd = datosImportados["yd"]

Inicializo los parámetros

In [0]:
CENTER = np.linspace(x.min(), x.max(), hidden_n)

# SIGMA(i) is the variance for i - th center
SIGMA = 0.02 * np.ones((hidden_n, 1))

W = np.zeros((hidden_n, out_n))

# Dejo sigma constante y los centros equidistantes
const_sigma = 0.26

MSE = np.zeros((max_epoch, 1)) # Root mean squared error
MCENTER = np.zeros((max_epoch,len(CENTER)))
MSIGMA = np.zeros((max_epoch, len(SIGMA)))
MW = np.zeros((max_epoch, len(SIGMA)))


phi = np.zeros(len(CENTER))
y = np.zeros(len(yd))
error = np.zeros(len(yd))

Calculo la red y grafico

In [0]:
nep = 1 # Número de época
objetivoAlcanzado = False
while ((nep < max_epoch) and (not objetivoAlcanzado)):
    for n in range(len(yd)): # Recorre todos los puntos x
        for i in range(len(CENTER)):
            phi[i] = np.exp(-(x[n] - CENTER[i])**2 / (2 * SIGMA[i]**2))
        y[n] = np.sum(W.transpose()*phi)
        error[n] = yd[n] - y[n]

        for i in range(len(CENTER)):
            CENTER[i] = CENTER[i] # + eta * error[0,n] * W[i] * phi[0,i] / SIGMA[i]**2 * (x[n] - CENTER[i])
            W[i] = W[i] + eta * error[n] * phi[i]
            SIGMA[i] = const_sigma #SIGMA[i] + eta * error[0,n] * W[i] * phi[0,i] * (x[n] - CENTER[i])**2 * 2 / SIGMA[i]**3

    if (nep == 20):
        f1 = plt.figure(1)
        plt.plot(x, yd, 'o-', label = 'yd')
        plt.plot(x, y, 'r.-', label = 'y')
        f1.legend(loc='upper right')
        plt.title('Pasada número 20')
        f1.show()

    MSE[nep] = (np.absolute(error)).mean()
    MCENTER[nep] = CENTER
    MW[nep] = W.transpose()
    MSIGMA[nep] = SIGMA.transpose()

    if MSE[nep] < error_goal:
        objetivoAlcanzado = True
        print('Alcanza el objetivo en la vuelta número: %d\n' % nep)
    nep = nep + 1

f2 = plt.figure(2)
plt.plot(x, yd, 'o-', label = 'yd')
plt.plot(x, y, 'r.-', label = 'y')
f2.legend(loc = 'upper right')
plt.title('Objetivo cumplido (sigma y centros fijos)')
f2.show()

f3, ax3 = plt.subplots(2, 2)

ax3[0, 0].plot(MSE[1: nep, -1])
ax3[0, 0].set_title('MSE')

ax3[0, 1].plot(MW[1: nep, -1])
ax3[0, 1].set_title('MW')

ax3[1, 0].plot(MCENTER[1:nep, -1])
ax3[1, 0].set_title('MCENTER')

ax3[1, 1].plot(MSIGMA[1:nep, -1])
ax3[1, 1].set_title('MSIGMA')

plt.gca().yaxis.set_minor_formatter(plt.NullFormatter())
plt.subplots_adjust(hspace=0.55, wspace=0.35)

plt.show()

# Grafico las phis
mphi = np.zeros((len(yd), len(CENTER)))
for n in range(len(yd)): # recorre todos los puntos x
    for i in range(len(CENTER)):
        mphi[n, i] = np.exp(-(x[n] - CENTER[i])**2 / (2 * SIGMA[i]**2))
    #y(n) = sum(W'.*phi)

f4 = plt.figure(4)

for i in range(len(CENTER)):
    plt.plot(x, mphi[:, i] * W[i])

plt.title('Aportes individuales')
plt.plot(x, yd, 'r.-')
f4.show()